# Part 1 of Coursera Capstone project. 

Scraping Toronto postcodes from Wikipedia, cleaning and aggregating data at postal code level

In [3]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup #HTML scraper

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Accessing and parsing Wikipedia page:

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

Locate table of interest and iterate through it to pull in cell values: 

In [5]:
postcode_table_scrape = soup.table.tbody.find_all('td') 
table_read = [] #One long list contains all cell values from the table

for item in postcode_table_scrape:
    table_read.append(item.text)

Breaking down long list to three short lists. One for each column.
(There must be an easier way of doing this)

In [6]:
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(table_read)):
    column_index = i%3
    if column_index == 0: 
        postcode.append(table_read[i])
    elif column_index == 1: 
        borough.append(table_read[i])
    else:
        neighbourhood.append(table_read[i].rstrip().strip(']'))

Assembling pandas dataframe from lists, and giving clean column headers

In [22]:
df_postcode = pd.DataFrame({
    'PostalCode': postcode,
    'Borough': borough,
    'Neighborhood': neighbourhood
    })

1. Dropping rows where Borough is 'Not assigned'. 
2. Replacing 'Not assigned' in Neighborhood with Borough name
3. Aggregating at PostalCode concatenating Neighbourhood names into one string.

In [24]:
#Removing where Borough is not assigned
df_postcode = df_postcode[df_postcode['Borough'] != 'Not assigned'].reset_index(drop=True)

#Replacing Not Assigned Neighbourhood with Boroug values.
for i in range(0, len(df_postcode)):
    if df_postcode['Neighborhood'].iloc[i] == 'Not assigned': 
        df_postcode['Neighborhood'].iloc[i] = df_postcode['Borough'].iloc[i]

#Concatenating Neighbourhood strings to bring dataframe to PostalCode level
df_compressed = pd.DataFrame(
        df_postcode.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" %', '.join(x))
).reset_index()
                                                                            

In [9]:
#Checking that strings haven't been truncated:
df_compressed.to_csv('output.csv')

In [26]:
df_compressed.shape

(103, 3)